Import Classes

In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import imgaug.augmenters as iaa
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras import layers, models
from keras import applications
import matplotlib.pyplot as plt

Defining Random Image Augmentation

In [2]:
random_augmenter = iaa.Sequential([
    # Horizontal Flip
    iaa.Fliplr(0.5),

    # Vertical Flip
    iaa.Flipud(0.5),

    # Multiply
    iaa.Multiply((0.8, 1.2)),

    # Linear Contrast
    iaa.LinearContrast((0.6, 1.4)),

    # Affine
    iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
               rotate=(-30,30),
               scale=(0.5, 1.5)),

    
])

Split Images and Labels into two different arrays with Augmentation and Pre-processing (imgaug) 

In [3]:
path = 'Dataset/'
dataset_path = os.listdir('Dataset')
image_size = 224
class_labels = []
images = []
labels = []


for label in dataset_path:
    data_path = path + str(label)
    filenames = [i for i in os.listdir(data_path)]

    for filename in filenames:
        img = cv2.imread(data_path + '/' + filename)
        img = cv2.resize(img, (image_size, image_size))
        
        # image augmentation x10 (random augmentations)
        for i in range(10):
            class_labels.append((label, str('dataset_path' + '/' + label) + '/' + filename))
            augmented_image = random_augmenter(image=img)
            images.append(augmented_image)
            labels.append(label)
            # cv2.imshow("Augmented Image", augmented_image)
        # cv2.waitKey(0)


dataframe = pd.DataFrame(data=class_labels, columns=['Labels', 'Image'])

images = np.array(images)
images = images.astype('float32')

Encoding the Labels + Splitting the dataframe to train-test sets (80-20)

In [4]:
df_labels = dataframe['Labels'].values
print(df_labels)

df_labelEncoder = LabelEncoder()
df_labels = df_labelEncoder.fit_transform(df_labels)
print(df_labels)

df_labels = df_labels.reshape(-1, 1)
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(df_labels)

images, Y = shuffle(images, Y, random_state=0)
train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.2, random_state=0)

['Cassava__Unidentified_1' 'Cassava__Unidentified_1'
 'Cassava__Unidentified_1' ... 'Sweet_Potato__Unidentified_1'
 'Sweet_Potato__Unidentified_1' 'Sweet_Potato__Unidentified_1']
[0 0 0 ... 1 1 1]


Create the EfficientNetB0 model

In [ ]:
NUM_CLASSES = len(dataset_path)
IMG_SIZE = image_size
size = (IMG_SIZE, IMG_SIZE)

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

outputs = applications.EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)

model = models.Model(inputs, outputs)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

Train model by 30 epochs

In [ ]:
hist = model.fit(train_x, train_y, epochs=30, verbose=2)

Exporting model

In [ ]:
import joblib
joblib.dump(model, 'Model_EfficientNetB0')